In [1]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

# Set to display all the columns in dataset

pd.set_option("display.max_columns", None)

# Import psql to run queries 

import pandasql as psql

ModuleNotFoundError: No module named 'pandas'

In [ ]:
hfdata = pd.read_csv(r"C:\Users\k.leelasaipavan\Desktop\project\hflights.csv",header=0)
hfdata_bk=hfdata.copy()

In [ ]:
hfdata.head()

In [ ]:
hfdata.info()

In [ ]:
hfdata.isnull().sum()

In [ ]:
#del hfdata['CancellationCode']
del hfdata['TailNum']

In [ ]:
from sklearn.impute import KNNImputer

imputer_knn = KNNImputer(missing_values=np.nan)

# Fill the missing values for 'Driver_Age'

hfdata['DepTime'] = imputer_knn.fit_transform(hfdata[['DepTime']])
hfdata['ArrTime'] = imputer_knn.fit_transform(hfdata[['ArrTime']])
#hfdata['TailNum'] = imputer_knn.fit_transform(hfdata[['TailNum']])

In [ ]:
from sklearn.impute import SimpleImputer

imputer_str = SimpleImputer(missing_values=np.nan, strategy='most_frequent', fill_value=None,
                            verbose=0,copy=True, add_indicator=False)
hfdata['ActualElapsedTime'] = imputer_str.fit_transform(hfdata[['ActualElapsedTime']])
hfdata['AirTime'] = imputer_str.fit_transform(hfdata[['AirTime']])
hfdata['ArrDelay'] = imputer_str.fit_transform(hfdata[['ArrDelay']])
hfdata['DepDelay'] = imputer_str.fit_transform(hfdata[['DepDelay']])
hfdata['TaxiIn'] = imputer_str.fit_transform(hfdata[['TaxiIn']])
hfdata['TaxiOut'] = imputer_str.fit_transform(hfdata[['TaxiOut']])



In [ ]:
hfdata.head(20)

In [ ]:
hfdata.info()

In [ ]:
# Use LabelEncoder to handle categorical data

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

hfdata['UniqueCarrier'] = LE.fit_transform(hfdata[['UniqueCarrier']])
hfdata['Origin'] = LE.fit_transform(hfdata[['Origin']])
hfdata['Dest'] = LE.fit_transform(hfdata[['Dest']])


In [ ]:
del hfdata['CancellationCode']

In [ ]:
hfdata.info()

In [ ]:
IndepVar = []
for col in hfdata.columns:
    if col != 'Cancelled':
        IndepVar.append(col)
TargetVar = 'Cancelled'
x=hfdata[IndepVar]
y=hfdata[TargetVar]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state = 42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = mmscaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = mmscaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled)

In [ ]:
HF_Results = pd.read_csv(r"C:\Users\k.leelasaipavan\Desktop\project\hf_results.csv",header=0)
HF_Results.head()

In [ ]:
# Build the Calssification models with Over Sampling and compare the results

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB


# Create objects of classification algorithms with default hyper-parameters

ModelLR = LogisticRegression()
ModelDC = DecisionTreeClassifier()
ModelRF = RandomForestClassifier()
ModelKN = KNeighborsClassifier()
ModelSVM = SVC(probability=True)
ModelET = ExtraTreesClassifier()
ModelGNB = GaussianNB()



# Evalution matrix for all the algorithms

#MM = [ModelLR, ModelDC, ModelRF, ModelET, ModelKNN, ModelGNB, ModelSVM, modelXGB, modelLGB]
MM = [ModelLR, ModelDC, ModelRF,ModelKN,ModelSVM,ModelET,ModelGNB]
for models in MM:
    
    # Assign values
    
    #x_train = xo_train
    #x_test = xo_test
    #y_train = yo_train
    #y_test = yo_test
            
    # Train the model training dataset
    
    models.fit(x_train, y_train)
    
    # Prediction the model with test dataset
    
    y_pred = models.predict(x_test)
    y_pred_prob = models.predict_proba(x_test)
    
    # Print the model name
    
    print('Model Name: ', models)
    
    # confusion matrix in sklearn

    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report

    # actual values

    actual = y_test

    # predicted values

    predicted = y_pred

    # confusion matrix

    matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
    print('Confusion matrix : \n', matrix)

    # outcome values order in sklearn

    tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
    print('Outcome values : \n', tp, fn, fp, tn)

    # classification report for precision, recall f1-score and accuracy

    C_Report = classification_report(actual,predicted,labels=[1,0])

    print('Classification report : \n', C_Report)

    # calculating the metrics

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    # Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
    # A model with a score of +1 is a perfect model and -1 is a poor model

    #from math import sqrt

    #mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    #MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

    print('Accuracy :', round(accuracy*100, 2),'%')
    print('Precision :', round(precision*100, 2),'%')
    print('Recall :', round(sensitivity*100,2), '%')
    print('F1 Score :', f1Score)
    print('Specificity or True Negative Rate :', round(specificity*100,2), '%'  )
    print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
    #print('MCC :', MCC)

    # Area under ROC curve 

    from sklearn.metrics import roc_curve, roc_auc_score

    print('roc_auc_score:', round(roc_auc_score(actual, y_pred), 3))
    
    # ROC Curve
    
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import roc_curve
    Model_roc_auc = roc_auc_score(actual, y_pred)
    fpr, tpr, thresholds = roc_curve(actual, models.predict_proba(x_test)[:,1])
    plt.figure()
    #
    plt.plot(fpr, tpr, label= 'Classification Model' % Model_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()
    print('-----------------------------------------------------------------------------------------------------')
    #----------------------------------------------------------------------------------------------------------
    new_row = {'Model Name' : models,
               'True Positive': tp,
               'False Negative': fn, 
               'False Positive': fp, 
               'True Negative': tn,
               'Accuracy' : accuracy,
               'Precision' : precision,
               'Recall' : sensitivity,
               'F1 Score' : f1Score,
               'Specificity' : specificity,
               'MCC': 'MCC',
               'ROC_AUC_Score':roc_auc_score(actual, y_pred),
               'Balanced Accuracy':balanced_accuracy}
    HF_Results = HF_Results.append(new_row, ignore_index=True)
    #----------------------------------------------------------------------------------------------------------

In [ ]:
HF_Results.head(10)

In [ ]:
HF_Results.head(10)